# Initialize
*(align with storyboard Version 6)*

In [ ]:
# Parameters.
# Times Square replaces this cell with the user's parameters.

# The run-time defaults for all of these parameters are in NightLog.yaml
# Under Times Square, the run-time defaults always override values given here.
# Values here are used for local tests.

# day_obs values: TODAY, YESTERDAY, YYYY-MM-DD
# Report on observing nights that start upto but not included this day.
#!day_obs = '2024-09-25' # Value to use for local testing (Summit)
day_obs = "2024-09-25"  # TODO Change to 'YESTERDAY' to test with default before push

# Total number of days of data to display (ending on day_obs)
number_of_days = "1"  # TODO Change to '1' to test with default before push

In [ ]:
import datetime as dt
import os
from collections import defaultdict
from pprint import pformat, pp
from urllib.parse import urlencode

import pandas as pd
import requests
from matplotlib import pyplot as plt

# When running under Times Square, install pkg from github.
# Otherwise use what is installed locally (intended to be dev editiable pkg)
if os.environ.get("EXTERNAL_INSTANCE_URL"):
    print(
        'Installing "lsst.ts.logging_and_reporting" from github using "prototype" branch....'
    )
    !pip install --upgrade git+https://github.com/lsst-ts/ts_logging_and_reporting.git@prototype >/dev/null
import lsst.ts.logging_and_reporting.almanac as alm
import lsst.ts.logging_and_reporting.reports as rep
from lsst.ts.logging_and_reporting.all_sources import AllSources

# import lsst.ts.logging_and_reporting.source_adapters as sad
import lsst.ts.logging_and_reporting.utils as ut
from lsst.ts.logging_and_reporting.reports import md, mdlist

try:
    import lsst.ts.logging_and_reporting.version

    lrversion = lsst.ts.logging_and_reporting.version.__version__
except:
    lrversion = "LIVE"

try:
    from lsst.summit.utils import ConsDbClient

    have_consdb = True
except:
    have_consdb = False

In [ ]:
# Normalize Parameters (both explicit Times Squares params, in implicit ones)
limit = 5000  # YAGNI for Auto get more if this isn't enough to get all requested DAYS
response_timeout = 3.05  # seconds, how long to wait for connection
read_timeout = 20  # seconds
timeout = (float(response_timeout), float(read_timeout))

date = ut.get_datetime_from_dayobs_str(day_obs)
# date:  is EXCLUSIVE (upto, but not including)
days = int(number_of_days)

# Thus: [min_day_obs,max_day_obs)
# Format: string, YYYY-MM-DD
min_date = date - dt.timedelta(days=days - 1)
max_date = date + dt.timedelta(days=1)
min_day_obs = min_date.strftime("%Y-%m-%d")  # Inclusive
max_day_obs = max_date.strftime("%Y-%m-%d")  # prep for Exclusive

In [ ]:
# Set default env to "usdf" and try before PUSH to repo.
summit = "https://summit-lsp.lsst.codes"
usdf = "https://usdf-rsp-dev.slac.stanford.edu"
tucson = "https://tucson-teststand.lsst.codes"

# The default provided here is for local testing.
# Under Times Square it is ignored.
server = os.environ.get(
    "EXTERNAL_INSTANCE_URL", summit
)  # TODO try with "usdf" before push (else "summit")

In [ ]:
# Read records from (almost) all sources
allsrc = AllSources(
    server_url=server,
    min_dayobs=min_day_obs,
    max_dayobs=max_day_obs,
)

nr_rep = rep.NightlyLogReport(adapter=allsrc.nig_src)
exposure_rep = rep.ExposurelogReport(adapter=allsrc.exp_src)
narrative_rep = rep.NarrativelogReport(adapter=allsrc.nar_src)

# Table of Contents
(TODO: update to reflect new content and ordering change)
* [Almanac](#almanac)
* [Summary plots of whole night](#Summary-Plots)
* [Summary Scalars for the night](#Summary-Scalars)
* [Night Report](#Night-Report)
* [Jira Tickets](#Jira-Tickets)
* [BLOCKS Observed](#BLOCKS-Observed)
* [Exposure Log](#Exposure-Log)
* [Narrative Log](#Narrative-Log)
* [Specific Plots](#Specific-Plots)
* [Developer Only](#dev-only)

In [ ]:
instrum_str = ", ".join(list(allsrc.exp_src.instruments.keys()))
md(
    f"# Showing data for **{min_date.date()}** to **{max_date.date()}** for {instrum_str}"
)

<a id="almanac"></a>
## Almanac

In [ ]:
# Display various almanac values (for moon, sun)
rep.AlmanacReport().day_obs_report(min_day_obs)

In [ ]:
# Load all the data sources
allsrc = AllSources(
    server_url=server,
    max_dayobs=max_day_obs,
    min_dayobs=min_day_obs,
)
nr_rep = rep.NightlyLogReport(adapter=allsrc.nig_src)

<a id="Summary-Plots"></a>
## Summary plots of whole night 

(content not yet defined in storyboard)

In [ ]:
# Exposure Tally
md('<a id="Summary-Scalars"></a>')
md(f"## Summary Scalars for the night ")
md("(all available instruments)")
tally = await allsrc.night_tally_observation_gaps()
if tally:
    display(pd.DataFrame(tally))

## Night Report 

In [ ]:
# NR Report

# Display time log
nr_rep.time_log_as_markdown()
md("-------------")
md("(TODO: better Jira tickets, BLOCKS observed)")


# Display Jira BLOCKS
front = "https://rubinobs.atlassian.net/projects/BLOCK?selectedItem=com.atlassian.plugins.atlassian-connect-plugin:com.kanoah.test-manager__main-project-page#!/"
tickets = allsrc.nig_src.nightly_tickets()
if tickets:
    mdstr = "#### Nightly Jira BLOCKs"
    for day, url_list in tickets.items():
        mdstr += f"\n- {day}"
        for ticket_url in url_list:
            mdstr += f'\n    - [{ticket_url.replace(front,"")}]({ticket_url})'
    md(mdstr)
else:
    md(f"No jira BLOCK tickets found using: [API Data]({allsrc.nig_src.source_url})")

## Exposure Log

In [ ]:
# Exposure Report

# Time Log
exposure_rep.time_log_as_markdown()
md("-------------")

# Observation gaps
gaps = allsrc.exp_src.get_observation_gaps()
if gaps:
    md(f"## Date vs Observation Gap (minutes) for all Instruments")
    for instrument, day_gaps in gaps.items():
        if len(day_gaps) == 0:
            md(f"**No day gaps found for *{instrument=!s}* **", color="lightblue")
        else:
            x, y = zip(*day_gaps.items())
            df = pd.DataFrame(dict(day=x, minutes=y))
            df.plot.bar(x="day", y="minutes", title=f"{instrument=!s}")
else:
    md(f"No Observation Gaps found.", color="lightblue")
    md(f"Used: [API Data]({allsrc.exp_src.source_url})")

In [ ]:
allsrc.min_dayobs, allsrc.max_dayobs, allsrc.server_url

In [ ]:
[len(recs) for recs in allsrc.exp_src.exposures.values()]

## Narrative Log


In [ ]:
# Narrative Report
# Time Log
narrative_rep.time_log_as_markdown()
md("-------------")

<a id="Specific-Plots"></a>
## Specific Plots 
(content not yet defined in storyboard)

<a id="dev-only"></a>
# Developer Only Section 
Contains stuff only expected to be useful to developers.

May also contain sections that have moved out of he user section because they are not defined in the Storyboard.

## Overview 


In [ ]:
# Display overview of Report context
md(
    f"""## Project-Wide Night(s) Report 
- Run on logs and databases from **{server}/**
- Report **{days} observing night(s)** with the last reported night starting on **{date}**.
- This report will include available data from noon **{min_date}** to noon **{max_date}**.
- Using ***Prototype* Logging and Reporting** Version: **{lrversion}**
"""
)

## This report uses the following data sources
- NightReport
- Exposurelog
- Narrativelog
- EFD
- ConsDB
- (DDV)
- (Almanac from Astroplan)

## DDV 

In [ ]:
DDV = (
    f"{server}/rubintv-dev/ddv/index.html"
    if "summit" in server
    else f"{server}/rubintv/ddv/index.html"
)
md(f"Access DDV part of RubinTV: {DDV}")

## Where was this run?

The environments that have everything needed to run this page are:

    https://summit-lsp.lsst.codes
    https://usdf-rsp-dev.slac.stanford.edu

However, Times Square does not run on the Summit. It does run on USDF-dev.


## Section overviews moved here

In [ ]:
# Night Report Overview
nr_rep.overview()
# Exposure Report Overview
exposure_rep.overview()
# Narrative Report Overview
narrative_rep.overview()

In [ ]:
# Conditionally display our current ability to connect to all needed endpoints.
if False and not os.environ.get("EXTERNAL_INSTANCE_URL"):
    md("## Dashboard")
    md("(This is not done when running under Times Square.)")
    %run ./dashboard.ipynb

## Finale

In [ ]:
print(f"Finished {str(dt.datetime.now())}")